In [1]:
# report future potential sotcks.

# 오늘부터 ~ 가능한 일자까지 예측치 에서 기준에 부합하느것
import pymysql
import datetime
from datetime import date, timedelta
import configparser

import telepot


LIMIT = 0.70
cf = configparser.ConfigParser()
cf.read('config.cfg')
                               
DB_IP = cf.get('db', 'DB_IP')
DB_USER = cf.get('db', 'DB_USER')
DB_PWD = cf.get('db', 'DB_PWD')
DB_SCH = cf.get('db', 'DB_SCH')
VALID_USER = cf.get('telegram', 'VALID_USER')
TOKEN = cf.get('telegram', 'TOKEN')



conn = pymysql.connect(host=DB_IP, user=DB_USER, password=DB_PWD, db=DB_SCH, charset='utf8mb4')
def get_potential_datas():
    cursor = conn.cursor()           
    cursor.execute("select max(analyzeAt), max(evaluate) from data.forecast;")
    result = cursor.fetchone()
    target_at = result[0] - timedelta(days=result[1])
    query = "select type, code, analyzeAt, potential, volume from data.forecast where analyzeAt > %s and potential > %s order by analyzeAt, code asc"    
    cursor.execute(query, (target_at, str(LIMIT)))
    return cursor.fetchall()

def send_msg(msg):
    if len(msg) is 0:
        return
    print(msg)
    bot = telepot.Bot(TOKEN)    
    bot.sendMessage(chat_id=VALID_USER, text= msg)
    
def report():
    msg = ''
    for data in get_potential_datas():
        date = data[2]
        code = data[1]
        potential = data[3]
        volume = data[4]
        msg += date.strftime("%Y-%m-%d") + ' [' + code + '] ['+ str(potential) + '] [' + str(volume) + ']\n'
    send_msg(msg)

In [2]:
import sys
import random
import traceback
import telepot
from telepot.delegate import per_chat_id, create_open, pave_event_space


class WorkBot(telepot.helper.ChatHandler):
    def __init__(self, *args, **kwargs):
        super(WorkBot, self).__init__(*args, **kwargs)
        
    
    def open(self, initial_msg, seed):
        self.sender.sendMessage('bot start')
        return True 

    def on_chat_message(self, msg):
        content_type, chat_type, chat_id = telepot.glance(msg)
        #{'date': 1489804035, 'text': 'dd', 'message_id': 2396, 'chat': {'type': 'private', 'first_name': 'YoungMin', 
        #'id': 60403284, 'last_name': 'Jung'}, 'from': {'first_name': 'YoungMin', 'id': 60403284, 'last_name': 'Jung'}}
        
        if msg['text'] == 'exit':
            self.sender.sendMessage('Bye!')
            self.close()
            return
        input_text = msg['text']
        # TODO -- name or code << send future potential.
        stock = get_stock_by_data(input_text)
        if stock is None:
            self.sender.sendMessage('not found.')
        poten_data = get_potential(stock)
        self.sender.sendMessage(poten_data)
    def get_stock_by_data(self, input_data):
        #TODO
    def get_potential(self, stock):
        #TODO
        
    def on__idle(self, event):
        self.sender.sendMessage('bot end.')
        self.close()



bot = telepot.DelegatorBot(TOKEN, [
    pave_event_space()(
        per_chat_id(), create_open, WorkBot, timeout=10),
])
bot.message_loop(run_forever='Listening ...')

Listening ...
{'date': 1489804035, 'text': 'dd', 'message_id': 2396, 'chat': {'type': 'private', 'first_name': 'YoungMin', 'id': 60403284, 'last_name': 'Jung'}, 'from': {'first_name': 'YoungMin', 'id': 60403284, 'last_name': 'Jung'}}


KeyboardInterrupt: 